In [1]:
import os
from collections import namedtuple
import json

In [ ]:
# Modality (01 = full-AV, 02 = video-only, 03 = audio-only).
# Vocal channel (01 = speech, 02 = song).
# Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).
# Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the 'neutral' emotion.
# Statement (01 = "Kids are talking by the door", 02 = "Dogs are sitting by the door").
# Repetition (01 = 1st repetition, 02 = 2nd repetition).
# Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).

In [2]:
Metadata = namedtuple('Metadata', 'modeality vocal emotion intensity statement repetition actor gender')

In [3]:
modality = {
    "01": "full-AV",
    "02": "video-only",
    "03": "audio-only",
}
vocal_channel = {
    "01": "speech",
    "02": "song",
}
emotion = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised",
}
intensity = {
    "01": "normal",
    "02": "strong",
}
statement = {
    "01": "Kids are talking by the door",
    "02": "Dogs are sitting by the door",
}

In [5]:
data_path = "/cantab/data/RAVDESS"

In [8]:
def parse_filename(filepath):
    filename = filepath.split("/")[-1].split(".")[0] # e.g. 03-01-04-02-02-02-02
    m, v, e, i, s, r, a = filename.split("-")
    
    m = modality[m]
    v = vocal_channel[v]
    e = emotion[e]
    i = intensity[i]
    s = statement[s]
    r = int(r)
    a = int(a)
    g = "male" if a % 2 == 1 else "female"
    
    meta_named = Metadata(m, v, e, i, s, r, a, g)
    meta_dict = {
        "modeality": m,
        "vocal_channel": v,
        "emotion": e,
        "intensity": i,
        "statement": s,
        "repetition": r,
        "actor": a,
        "gender": g,
    }
    
    return meta_dict, meta_named 

In [14]:
file_dict = {}
file_meta = {}
for subdir, dirs, files in os.walk(data_path):
    for file in files:
        file_path = os.path.join(subdir, file)

        if file_path.endswith(".wav"):
            meta_dict, meta_named  = parse_filename(file_path)
            file_dict[file_path] = meta_dict
            file_meta[file_path] = meta_named

In [11]:
with open('emotion/metadata.json', 'w') as fp:
    json.dump(file_dict, fp, indent=4)

In [16]:
emotion2class = {
    "neutral": 0,
    "calm": 1,
    "happy": 2,
    "sad": 3,
    "angry": 4,
    "fearful": 5,
    "disgust": 6,
    "surprised": 7,
}

with open('emotion/all.dbl', 'w') as fp:
    for filepath, meta in file_meta.items():
        fp.write(f"{filepath} {emotion2class[meta.emotion]}\n")